# FingerPrint Classification
__Author__ : Mohammad Rouintan , 400222042

__Course__ : Undergraduate Digital Image Processing Course

In [351]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
import glob
import os

In [413]:
path = '/Users/mohammad/Downloads/NISTSpecialDatabase4GrayScaleImagesofFIGS/sd04/png_txt'
imgs = glob.glob(path + '/**/*.png', recursive=True)
txts = glob.glob(path + '/**/*.txt', recursive=True)

In [414]:
def pre_process(img):
    # Read Image
    img_gray = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img_gray = cv2.GaussianBlur(img_gray, (7, 7), 0)

    # Convert To Binary
    img_binary = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 13, 4)
    
    # Morphological preprocess
    B = np.ones((3, 3), np.uint8)
    img_morph = cv2.morphologyEx(cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, B), cv2.MORPH_CLOSE, B)


    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    processed_image = cv2.morphologyEx(img_morph, cv2.MORPH_OPEN, kernel)

    return processed_image

In [415]:
p_imgs = []
for i in range(len(imgs)):
    img = pre_process(imgs[i])
    p_imgs.append(img)

In [425]:
archs, tented_archs, left_loops, right_loops, whorls = [], [], [], [], []
for i in range(1000, len(txts)-1000):
    f = open(txts[i], 'r')
    f.readline()
    char = f.readline()[-2]
    if char == 'A' and len(archs) < 20:
        archs.append(p_imgs[i])
    elif char == 'T' and len(tented_archs) < 20:
        tented_archs.append(p_imgs[i])
    elif char == 'L' and len(left_loops) < 20:
        left_loops.append(p_imgs[i])
    elif char == 'R' and len(right_loops) < 20:
        right_loops.append(p_imgs[i])
    elif char == 'W' and len(whorls) < 20:
        whorls.append(p_imgs[i])

In [131]:
def find_gradient(img):
    img = cv2.resize(img, (50, 50))

    gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
    gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT) 


    # gradient_magnitude = np.sqrt(gx ** 2 + gy ** 2)
    # gradient_theta = np.arctan((-2*gx*gy)/(gx ** 2 - gy ** 2))
    # print(gradient_magnitude)
    # print(gradient_theta)
    gradient = np.sqrt(gy**2 + gx**2)
    theta = np.arctan((-2*gx*gy)/(gx ** 2 - gy ** 2))
    plt.figure(figsize=(7, 7))
    plt.imshow(theta, cmap='gray')
    plt.show()
    plt.figure(figsize=(7, 7))
    plt.imshow(gradient, cmap='gray')
    plt.show()

    # theta = cv2.resize(theta, (150, 150))
    
    plt.figure(figsize=(20, 10))
    plt.imshow(img, cmap='gray')
    plt.quiver(range(img.shape[0]), range(img.shape[1]), gx, gy, color='green',scale_units='width')
    plt.show()

In [417]:
from skimage.feature import hog
def get_hog(img):
    hog_features = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return hog_features

In [418]:
h_imgs = []
for img in p_imgs:
    h_imgs.append(get_hog(img))

In [426]:
h_archs = [get_hog(item) for item in archs]
h_tented_archs = [get_hog(item) for item in tented_archs]
h_left_loops = [get_hog(item) for item in left_loops]
h_right_loops = [get_hog(item) for item in right_loops]
h_whorls = [get_hog(item) for item in whorls]

In [420]:
from scipy.spatial import distance

In [421]:
def classifier(img):
    maximum = 0
    category = None

    for arch in h_archs:
        similarity = 1 - distance.cosine(img, arch)
        if similarity >= maximum:
            maximum = similarity
            category = 'A'

    for tented_arch in h_tented_archs:
        similarity = 1 - distance.cosine(img, tented_arch)
        if similarity >= maximum:
            maximum = similarity
            category = 'T'


    for left_loop in h_left_loops:
        similarity = 1 - distance.cosine(img, left_loop)
        if similarity >= maximum:
            maximum = similarity
            category = 'L'

    for right_loop in h_right_loops:
        similarity = 1 - distance.cosine(img, right_loop)
        if similarity >= maximum:
            maximum = similarity
            category = 'R'

    for whorl in h_whorls:
        similarity = 1 - distance.cosine(img, whorl)
        if similarity >= maximum:
            maximum = similarity
            category = 'W'

    return category

In [427]:
labels = []
for img in h_imgs:
    labels.append(classifier(img))

In [423]:
true_labels = []
for txt in txts:
    f = open(txt, 'r')
    f.readline()
    true_labels.append(f.readline()[-2])

In [428]:
print(labels)
print(true_labels)

['L', 'T', 'R', 'R', 'W', 'R', 'T', 'L', 'R', 'L', 'L', 'L', 'T', 'W', 'L', 'W', 'R', 'A', 'W', 'T', 'T', 'L', 'L', 'W', 'R', 'R', 'L', 'L', 'W', 'R', 'A', 'L', 'W', 'R', 'A', 'L', 'W', 'R', 'L', 'L', 'R', 'T', 'L', 'W', 'W', 'W', 'R', 'L', 'A', 'W', 'L', 'L', 'T', 'W', 'W', 'T', 'R', 'A', 'R', 'L', 'T', 'W', 'W', 'L', 'W', 'T', 'W', 'T', 'T', 'T', 'W', 'L', 'R', 'W', 'W', 'T', 'W', 'W', 'W', 'W', 'W', 'T', 'L', 'L', 'L', 'R', 'W', 'L', 'L', 'W', 'L', 'T', 'L', 'W', 'W', 'A', 'R', 'T', 'L', 'R', 'T', 'L', 'T', 'W', 'L', 'A', 'R', 'L', 'W', 'R', 'T', 'L', 'W', 'L', 'W', 'L', 'L', 'T', 'R', 'T', 'L', 'W', 'L', 'R', 'L', 'R', 'L', 'T', 'W', 'T', 'T', 'L', 'R', 'R', 'T', 'R', 'R', 'L', 'L', 'T', 'R', 'L', 'R', 'T', 'W', 'W', 'L', 'R', 'A', 'T', 'A', 'A', 'L', 'A', 'W', 'A', 'W', 'L', 'T', 'T', 'R', 'R', 'T', 'L', 'T', 'W', 'L', 'W', 'W', 'T', 'R', 'W', 'L', 'L', 'W', 'W', 'L', 'T', 'R', 'T', 'R', 'T', 'W', 'L', 'W', 'L', 'W', 'L', 'T', 'L', 'R', 'L', 'W', 'L', 'T', 'R', 'L', 'R', 'L', 'T',

In [430]:
acc = 0
for i in range(4000):
    if labels[i] == true_labels[i]:
        acc += 1

print(acc/4000 * 100)

22.2
